# Amsi-Bypass-Powershell
git@github.com:S3cur3Th1sSh1t/Amsi-Bypass-
Powershell.git

## extract ps1 from readme.md

In [2]:
import os
import re

# ====== Config ======
input_file = './Amsi-Bypass-Powershell/README.md'            
output_dir = './Amsi-Bypass-Powershell/generated_ps1'         
os.makedirs(output_dir, exist_ok=True)

# ====== Read README content ======
with open(input_file, 'r', encoding='utf-8') as f:
    content = f.read()

# ====== Find all powershell code blocks ======
pattern = re.compile(r'```powershell\s*\n(.*?)```', re.DOTALL | re.IGNORECASE)
matches = pattern.findall(content)

print(f"✅ Tổng số đoạn script tìm được: {len(matches)}")

# ====== Process and save each script ======
valid_script_count = 0

for idx, script in enumerate(matches, 1):
    script = script.strip()

    # Bỏ qua những đoạn quá ngắn (không đáng để lưu)
    if len(script) < 10:
        continue

    # Nếu trong script có comment ghi filename, lấy luôn
    filename_match = re.search(r'#\s*Filename\s*:\s*(.+?\.ps1)', script, re.IGNORECASE)
    if filename_match:
        filename = filename_match.group(1).strip()
    else:
        filename = f'bypass_{idx:03d}.ps1'

    output_path = os.path.join(output_dir, filename)

    # Ghi file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(script)

    valid_script_count += 1
    print(f"✅ Saved {output_path}")

print(f"\n🎯 Hoàn thành: Đã tách được {valid_script_count} script hợp lệ!")

✅ Tổng số đoạn script tìm được: 10
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_001.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_002.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_003.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_004.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_005.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_006.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_007.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_008.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_009.ps1
✅ Saved ./Amsi-Bypass-Powershell/generated_ps1/bypass_010.ps1

🎯 Hoàn thành: Đã tách được 10 script hợp lệ!


## convert ps1 -> csv

In [4]:
import os
import csv

# ====== Config ======
input_folder = './Amsi-Bypass-Powershell/generated_ps1'  # Folder chứa các .ps1
output_csv = './dataset/Amsi-Bypass-Powershell.csv'

# ====== Build payloads ======
data_list = []

for filename in os.listdir(input_folder):
    if filename.endswith('.ps1'):
        file_path = os.path.join(input_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            b64_decoded = f.read().strip()

        # === Áp dụng giống logic combine_attributes ===
        path_exec = "powershell"
        arguments = ""  # Không có arguments
        full_payload = f"{path_exec} {arguments} {b64_decoded}".strip()

        data = {
            'Payload': full_payload,
            'Label': 'Bypass'
        }
        data_list.append(data)

# ====== Save CSV ======
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Payload', 'Label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for data in data_list:
        writer.writerow(data)

print(f"✅ Đã lưu {len(data_list)} samples vào {output_csv}")

✅ Đã lưu 10 samples vào ./dataset/Amsi-Bypass-Powershell.csv


# mpsd

In [ ]:
import os
import hashlib
import time
import pandas as pd
import requests
from dotenv import load_dotenv

# ===== Cấu hình =====
input_dir = './mpsd/malicious_pure'
output_csv = './dataset/mpsd.csv'
unknown_csv = './mpsd/label_unknown.csv'
checkpoint_file = './mpsd/labeled_sha256.txt'

load_dotenv()
api_keys = [os.getenv(f'api{i}') for i in range(1, 10) if os.getenv(f'api{i}')]

# ===== Hàm hỗ trợ =====
def get_sha256(content: bytes) -> str:
    return hashlib.sha256(content).hexdigest()

def map_label_from_vt(vt_name: str, categories: list) -> str:
    name = (vt_name or "").lower()
    cats = [c.lower() for c in categories] if categories else []

    BYPASS_KEYWORDS = ['bypass', 'disable defender', 'evade', 'avoid', 'kill defender', 'anti-av']
    TASKEXEC_KEYWORDS = ['schtask', 'task', 'scheduled', 'com', 'wmi', 'bitsadmin', 'bits', 'vb task', 'dynamite']
    DOWNLOADER_KEYWORDS = ['downloader', 'dropper', 'transfer', 'webdl', 'fetch', 'grabber', 'iexds', 'dfsp', 'curl', 'wget', 'powershellhttp']
    INJECTOR_KEYWORDS = ['inject', 'shellcode', 'reflective', 'pe loader', 'runpe', 'rhttp', 'remote dll', 'trojan', 'hollows', 'process hollowing']
    PAYLOAD_KEYWORDS = ['empire', 'veil', 'power', 'txt c2', 'powershell backdoor', 'remcos', 'reverse shell']

    def match(keywords):
        return any(kw in name for kw in keywords) or any(kw in c for kw in keywords for c in cats)

    if match(BYPASS_KEYWORDS):
        return 'Bypass'
    elif match(TASKEXEC_KEYWORDS):
        return 'TaskExecution'
    elif match(DOWNLOADER_KEYWORDS):
        return 'Downloader'
    elif match(INJECTOR_KEYWORDS):
        return 'Injector'
    elif match(PAYLOAD_KEYWORDS):
        return 'Payload'
    return 'Unknown'

# ===== Checkpoint =====
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        labeled_set = set(line.strip() for line in f)
else:
    labeled_set = set()

# ===== Khởi tạo =====
ps1_files = [f for f in os.listdir(input_dir) if f.endswith('.ps1')]
results = []
unknowns = []
api_index = 0

# ===== Xử lý từng file =====
for fname in ps1_files:
    fpath = os.path.join(input_dir, fname)
    with open(fpath, 'rb') as f:
        content_bytes = f.read()

    sha256 = get_sha256(content_bytes)
    if sha256 in labeled_set:
        continue

    content_str = content_bytes.decode('utf-8', errors='ignore')
    current_key = api_keys[api_index]
    api_index = (api_index + 1) % len(api_keys)

    url = f"https://www.virustotal.com/api/v3/files/{sha256}"
    headers = {"x-apikey": current_key}

    try:
        r = requests.get(url, headers=headers)
        if r.status_code == 404:
            label = 'Unknown'
        elif r.status_code == 403:
            print(f"❌ API key bị giới hạn: {current_key}")
            break
        elif r.status_code != 200:
            print(f"⚠️ Lỗi khi gọi {sha256}: {r.status_code}")
            continue
        else:
            data = r.json()
            name = data.get('data', {}).get('attributes', {}).get('meaningful_name', '')
            tags = data.get('data', {}).get('attributes', {}).get('tags', [])
            label = map_label_from_vt(name, tags)

    except Exception as e:
        print(f"⚠️ Lỗi khi gọi VT: {e}")
        continue

    row = {
        'SHA256': sha256,
        'Full Payload': content_str,
        'Label': label
    }

    # Ghi checkpoint
    with open(checkpoint_file, 'a') as f:
        f.write(sha256 + '\n')

    row_df = pd.DataFrame([row])

    if label == 'Unknown':
        row_df.to_csv(unknown_csv, mode='a', header=not os.path.exists(unknown_csv), index=False)
    else:
        row_df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

    print(f"✅ {fname} → {label}")
    time.sleep(2)

# ===== Lưu kết quả =====
if results:
    df = pd.DataFrame(results)
    if os.path.exists(output_csv):
        df.to_csv(output_csv, mode='a', header=False, index=False)
    else:
        df.to_csv(output_csv, index=False)
    print(f"✅ Đã lưu {len(results)} nhãn hợp lệ → {output_csv}")

if unknowns:
    df = pd.DataFrame(unknowns)
    if os.path.exists(unknown_csv):
        df.to_csv(unknown_csv, mode='a', header=False, index=False)
    else:
        df.to_csv(unknown_csv, index=False)
    print(f"ℹ️ Đã lưu {len(unknowns)} mẫu Unknown → {unknown_csv}")

if not results and not unknowns:
    print("👍 Không có file mới cần xử lý.")


✅ 3144.ps1 → Payload
✅ 3165.ps1 → Payload
✅ 2853.ps1 → Payload
✅ 247.ps1 → Unknown
✅ 537.ps1 → Unknown
✅ 2733.ps1 → Unknown
✅ 3182.ps1 → Payload
✅ 2856.ps1 → Payload
✅ 3374.ps1 → Payload
✅ 309.ps1 → Payload
✅ 1117.ps1 → Unknown
✅ 1616.ps1 → Unknown
✅ 2255.ps1 → Payload
✅ 1400.ps1 → Unknown
✅ 1653.ps1 → Unknown
✅ 3419.ps1 → Unknown
✅ git_45.ps1 → Payload
✅ 3818.ps1 → Unknown
✅ 3663.ps1 → Unknown
✅ 428.ps1 → Unknown
✅ 216.ps1 → Payload
✅ 3751.ps1 → Unknown
✅ 1407.ps1 → Payload
✅ 3754.ps1 → Unknown
✅ 1768.ps1 → Payload
✅ 1194.ps1 → Unknown
✅ 3267.ps1 → Payload
✅ 881.ps1 → Payload
✅ 201.ps1 → TaskExecution
✅ 2410.ps1 → Payload
✅ 2302.ps1 → TaskExecution
✅ 764.ps1 → Payload
✅ 1786.ps1 → Payload
✅ 1409.ps1 → Payload
✅ 1345.ps1 → Unknown
✅ 3090.ps1 → Payload
✅ 2409.ps1 → Payload
✅ 2656.ps1 → Unknown
✅ 1945.ps1 → Payload
✅ 1046.ps1 → Payload
✅ 1120.ps1 → Payload
✅ 880.ps1 → Unknown
✅ 3269.ps1 → Payload
✅ 964.ps1 → Unknown
✅ 3498.ps1 → Unknown
✅ 2920.ps1 → Unknown
✅ 2549.ps1 → Payload
✅ 2604.ps